# DSS-Commander (DSS Results Plotting with Calibration Statistics)
Authors: <br>William (Bill) Katzenmeyer, P.E., C.F.M. (C.H. Fenstermaker and Associates, LLC) 
<br>         Tyler Young, E.I. (Calibration Metrics Calculations, Debugging and Testing)

Source: https://github.com/billk-FM/HEC-Commander-Tools

In [ ]:
# USER INPUTS: 

# DSS and Plotting Parameters
DSS_Source_Path = r"C:\Path\To_Your\DSS Results"                                            # DSS Source Path with DSS files to plot   
gauge_csv_file_name = "Gauge_Data.csv"                                                      # Gauge data file name (within root folder)     
river_stations = ["12345", "6789", "101112"]                                                # Subbasin names can also be used to analyze HEC-HMS results
search_word = "March"                                                                       # Enter a word from your HMS or RAS run name to filter results
plot_window_start , plot_window_end = "01APR2021 00:00:00" , "10APR2021 00:00:00"           # Example: "28MAR2018 00:00:00" , "10APR2018 00:00:00"   
base_condition_dss = "EVENT_BASE"                                                           # Time series matching this search phrase will be plotted in blue


# Time Series Type: ex. "STAGE" or "FLOW"
search_parameter1 = "STAGE"
search_parameter2 = "FLOW"

# PLOT CUSTOMIZATION
# Customization Options:
bokeh_plot_width = 1250
bokeh_plot_height = 600
bokeh_table_width = 1250
bokeh_table_height = 200

# File Names
output_html_file_name = "_Plots.html"
bokeh_plot_title = "Time Series Plots by River Station and Series Type"

# Additional Information 
# This is the bottom elevation of the river_stations mentioned above, so depth can be calculated (in the same order as the river_stations)
chan_bottom_elevation_dict = {"89804": 112.3, "117188": 79.25, "110449": 43.66}



'''For best results, ensure the gauge data and dss files fully cover the plot window.  '''

In [ ]:
'''Example Gauge Data CSV File:
River Station,89804,89804,117188,110449,110449
STAGE or FLOW,STAGE,FLOW,STAGE,STAGE,FLOW
Date,USGS_STA_NUM1,USGS_STA_NUM1,USGS_STA_NUM2,USGS_STA_NUM3,USGS_STA_NUM3
3/28/2018 0:00,116.8,60.7,95.195,50.146,371
3/28/2018 1:00,116.8,60.7,95.195,50.126,368
3/28/2018 2:00,116.79,60.1,95.195,50.136,369

'''

In [ ]:
# Import required Libraries
import subprocess
import sys
import platform

def install_module(module_name):
    try:
        __import__(module_name)
        #print(f"{module_name} is already installed.")
    except ImportError:
        print(f"{module_name} not found. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", module_name])

import sys

def python_version_check(version):
    current_version = tuple(map(int, sys.version_info[:3]))
    required_version = tuple(map(int, version.split(".")))
    if current_version < required_version:
        print(f"Current Python version ({'.'.join(map(str, current_version))}) is lower than the required version ({version}).")
        print("Please create an environment with Python version", version, "or higher.")
    else:
        print(f"Current Python version ({'.'.join(map(str, current_version))}) meets the required version ({version}).")

# Check Python version
python_version_check("3.9.1")


def download_and_install_wheel(wheel_url):
    import requests 
    wheel_file = wheel_url.split("/")[-1]
    response = requests.get(wheel_url)
    
    if response.status_code == 200:
        with open(wheel_file, 'wb') as f:
            f.write(response.content)
        subprocess.check_call([sys.executable, "-m", "pip", "install", wheel_file])
        print(f"Successfully installed GDAL from {wheel_file}.")
    else:
        print(f"Failed to download {wheel_file}. HTTP Status Code: {response.status_code}")



def install_osgeo_and_pydss():
    # Check and install osgeo
    try:
        from osgeo import ogr
        print("Successfully imported ogr from osgeo.")
    except ImportError:
        print("Failed to import ogr from osgeo. Attempting to download and install GDAL wheel...")

        # Get Python version and system architecture
        python_version = f"cp{sys.version_info.major}{sys.version_info.minor}"
        arch = 'win_amd64' if platform.architecture()[0] == '64bit' else 'win32'

        # Generate the wheel URL dynamically
        wheel_url = f"https://download.lfd.uci.edu/pythonlibs/archived/GDAL-3.4.3-{python_version}-{python_version}-{arch}.whl"

        download_and_install_wheel(wheel_url)

        # Re-try importing the ogr module
        try:
            from osgeo import ogr
            print("Successfully imported ogr from osgeo.")
        except ImportError:
            print("Still unable to import ogr from osgeo after attempting to install. Please check your environment.")

    # Check and install PyDSS
    try:
        from pydsstools.heclib.dss import HecDss
    except ImportError:
        install_module("pydsstools")
        print("PyDSS not found. If this fails repeatedly, please follow the steps below for all dependencies:")
        print("1. Install essential dependencies: NumPy, pandas, affine, bokeh, chardet")
        print("2. Install C++ Build Tools for Visual Studio 2022 from:")
        print("   https://download.visualstudio.microsoft.com/download/pr/33081bfc-10f1-42d4-8f5a-df6709b8b105/5ae40ab45b17219f8a9c505f5106bbe818bceba5f8e92bd1df5d6db8fa44d820/vs_BuildTools.exe")
        print("3. Download and install the PyDSS Wheel from:")
        print("   https://raw.githubusercontent.com/gyanz/pydsstools/master/dist/pydsstools-2.2-cp39-cp39-win_amd64.whl")


# List of modules to check and install if necessary
modules = ['affine',
 'rasterio',
 'tables',
 'pandas',
 'shutil',
 'numba',
 'numpy',
 'cython',
 'requests',
 'bs4',
 'chardet',
 'concurrent.futures',
 'csv',
 'rioxarray',
 'xarray',
 'pyproj',
 'platform',
 'scikit-learn',
 'openpyxl',
 'bokeh',
 'os',
 'matplotlib',
 'shapely']

for module in modules:
    install_module(module)

# To execute the function:
install_osgeo_and_pydss()

from pydsstools.heclib.dss import HecDss
from pydsstools.heclib.dss.HecDss import Open
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import os
import shutil
from IPython.display import clear_output
import pyproj
import io
import os
import pandas as pd
import numpy as np
import sys
from numba import jit
import numpy as np
from datetime import datetime
from bokeh.palettes import Category20
from bokeh.plotting import figure, save, gridplot
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, Legend, LegendItem, HoverTool
from sklearn.metrics import mean_squared_error
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Font, Alignment
from openpyxl.formatting.rule import CellIsRule
from bokeh.models import DataTable, DateFormatter, TableColumn, ColumnDataSource, NumberFormatter, BooleanFormatter, StringFormatter, DateFormatter, CustomJS, ImageURL, HTMLTemplateFormatter 
from bokeh.layouts import layout
from bokeh.io import show, save
from bokeh.layouts import row
from bokeh.models.widgets import DataTable, DateFormatter, TableColumn, StringEditor, IntEditor, NumberEditor, SelectEditor, Div



In [ ]:
# Clip Gauge Data to Plot Window

# Define pathname_pattern
pathname_pattern = "/*/*/*/*/*/*/"

# Define the date range
plot_start_date = pd.to_datetime(plot_window_start, format='%d%b%Y %H:%M:%S')
plot_end_date = pd.to_datetime(plot_window_end, format='%d%b%Y %H:%M:%S')

# Function to get the date range from the first DSS file
# NEED TO INSERT HERE

# read the gauge data CSV file, filter the data based on the date range, and write the filtered data to a new CSV file
# This ensures that dates are aligned between the gauge data and DSS files

# Read the header rows
header_rows = pd.read_csv(os.path.join(DSS_Source_Path, gauge_csv_file_name), nrows=2)
# Remove suffix from duplicate column names in header rows
header_rows.columns = header_rows.columns.str.replace(r"\.\d+$", "", regex=True)
# Read the data, setting the header to the third row (0-indexed)
df = pd.read_csv(os.path.join(DSS_Source_Path, gauge_csv_file_name), skiprows=3)
# Rename the first column to 'Date'
df = df.rename(columns={df.columns[0]: 'Date'})
# Remove suffix from duplicate column names
df.columns = df.columns.str.replace(r"\.\d+$", "", regex=True)
# Ensure the 'Date' column is in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y %H:%M', errors='coerce')
# Drop rows with NaT in 'Date' column
df = df.dropna(subset=['Date'])
# Filter the DataFrame based on the date range
df = df[(df['Date'] >= plot_start_date) & (df['Date'] <= plot_end_date)]
# Write the header rows to the new CSV file
header_rows.to_csv(os.path.join(DSS_Source_Path, '00_Gauge_data_plot_window.csv'), index=False)
# Append the filtered DataFrame to the new CSV file, without writing the header
df.to_csv(os.path.join(DSS_Source_Path, '00_Gauge_data_plot_window.csv'), mode='a', index=False, header=False)

# Re-define gauge CSV file name to point to the filtered file
gauge_csv_file_name = '00_Gauge_data_plot_window.csv'

print(f"Plotting data from {plot_window_start} to {plot_window_end}")

In [ ]:
# Define HTML Templates
rmse_template = """
            <div style="color:<%= 
                (function colorfromint(){
                    if(value >= 0 && value <= 15)
                        return('green');
                    if(value > 15.0001 && value <= 100)
                        return('red');                        
                }()) %>; 
                font-weight: bold; 
                text-align: left;
            "><%= value.toFixed(2) %></div>
        """
r_template = """
    <div style="color:<%= 
        (function colorfromint(){
            if(value >= 0.9 && value <= 1)
                return('green');
            if(value < 0.9)
                return('red');
        }()) %>; 
        font-weight: bold; 
        text-align: left;
    "><%= value.toFixed(2) %></div>
"""
PBIAS_template = """
    <div style="color:<%= 
        (function colorfromint(){
            if(value >= -10 && value <= 10)
                return('green');
            if(value < -10)
                return('red');
            if(value > 10)
                return('red');
        }()) %>; 
        font-weight: bold; 
        text-align: left;
    "><%= value.toFixed(2) %></div>
"""
NSE_template = """
    <div style="color:<%= 
        (function colorfromint(){
            if(value >= 0.6 && value <= 1)
                return('green');
            if(value >= 0 && value < 0.6)
                return('red');
        }()) %>; 
        font-weight: bold; 
        text-align: left;
    "><%= value.toFixed(2) %></div>
"""

In [ ]:
# Define All Remaining Functions

# Define pathname_pattern
pathname_pattern = "/*/*/*/*/*/*/"

def adjust_dataframe(df, river_station, parameter):
    if parameter == 'STAGE' and river_station in chan_bottom_elevation_dict:
        # Adjust all columns, excluding the timestamp
        for col in df.columns[0:]:
            df[col] -= chan_bottom_elevation_dict[river_station]
    return df

def write_data_to_excel(grouped_data, folder_path, file_name, adjust=False):
    file_path = os.path.join(folder_path, file_name)
    with pd.ExcelWriter(file_path, engine="xlsxwriter") as writer:
        for key in grouped_data:
            river_station = key[0]
            parameter = key[1]
            sheet_name = f"{river_station}_{parameter}"
            
            # Rearrange the columns, so that Gauge_Data.csv is first (if available)
            gauge_data_columns = [series for series in grouped_data[key] if gauge_csv_file_name in series.name]
            other_columns = [series for series in grouped_data[key] if gauge_csv_file_name not in series.name]
            
            if len(gauge_data_columns) < 1:
                no_gauge_series = pd.Series(name="No gauge", dtype='float64')
                gauge_data_columns = [no_gauge_series]

            reordered_columns = gauge_data_columns + other_columns
            df_concat = pd.concat(reordered_columns, axis=1)
            
            if adjust:
                df_concat = adjust_dataframe(df_concat, river_station, parameter)
                
            df_concat.to_excel(writer, sheet_name=sheet_name)


# Update the two original functions to use the new write_data_to_excel function
def save_plotted_data_to_excel(grouped_data, folder_path):
    write_data_to_excel(grouped_data, folder_path, "1_Output_Plotted_Data.xlsx")

def save_plotted_data_to_excel_depth(grouped_data, folder_path):
    write_data_to_excel(grouped_data, folder_path, "1_Output_Plotted_Data_Depth.xlsx", adjust=True)




def check_identical_values(df, new_series):
    for column in df.columns:
        if np.array_equal(df[column].head(100).values, new_series.head(100).values):
            return True
    return False

def get_gauge_labels(grouped_data, gauge_csv_file_name):
        gauge_labels = {}
        for key in grouped_data:
            for series in grouped_data[key]:
                if gauge_csv_file_name in series.name:
                    river_station = key[0]
                    parameter = key[1]
                    label_name = series.name.replace(f"{gauge_csv_file_name}_", "").replace(".1", "")
                    gauge_labels[(river_station, parameter)] = label_name
        return gauge_labels


def calculate_stats(df_sheet, parameter): 
        # Remove any rows with empty observed data
        df_sheet = df_sheet[~df_sheet.iloc[:, 1].isnull()]

        # Check if the data is non-empty
        if df_sheet.empty:
            stats = []
        else:
            stats = []
            for col in df_sheet.columns[2:]:
                n = len(df_sheet)
                xi_mean = df_sheet.iloc[:, 1].mean()
                xi_peak = df_sheet.iloc[:, 1].max()
                yi = df_sheet[col].mean()

                rmse = np.sqrt(mean_squared_error(df_sheet.iloc[:, 1], df_sheet[col]))
                
                # Determine the divisor for rmse_percent based on parameter
                if parameter == 'STAGE':
                    divisor = xi_mean
                elif parameter == 'FLOW':
                    divisor = xi_peak  # Note: xi_peak needs to be defined or passed to this function
                else:
                    raise ValueError("Unknown parameter: {}".format(parameter))

                rmse_percent = (rmse / divisor) * 100  # Calculate percentage RMSE

                
                
                
                r = np.corrcoef(df_sheet.iloc[:, 1], df_sheet[col])[0][1]

                pbias = ((df_sheet.iloc[:, 1] - df_sheet[col]).sum() / df_sheet.iloc[:, 1].sum()) * 100  # Updated PBIAS formula

                nse = 1 - ((df_sheet.iloc[:, 1] - df_sheet[col])**2).sum() / ((df_sheet.iloc[:, 1] - xi_mean)**2).sum()

                col_modified = col.split('/')[:4]    # Split the column
                stats.append(col_modified + [n, rmse_percent, r, pbias, nse])  # Replace rmse with rmse_percent
        
        return stats


def plot_grouped_data_bokeh(grouped_data, folder_path, directory_name):
    output_file(os.path.join(folder_path, f"{directory_name}_Plots.html"), title=bokeh_plot_title)
    plots = []
    gauge_labels = get_gauge_labels(grouped_data, gauge_csv_file_name)

    # Load statistics data
    stats_data = pd.read_excel(os.path.join(folder_path, '2_Output_Statistics_Depth.xlsx'), sheet_name=None)  

    # Table for the "global summaries" data
    if "global summaries" in stats_data:
        global_summaries = stats_data["global summaries"]
        # Cleaning the data by dropping unnamed columns
        cleaned_data = global_summaries.drop(columns=global_summaries.columns[:4])
        global_src = ColumnDataSource(cleaned_data)
        # Use columns from the 'global summaries' 
        global_columns = [
                TableColumn(field="Simulation", title="Simulation",
                            formatter=StringFormatter(font_style="bold", text_align='left')),
                TableColumn(field="RMSE [%]", title="RMSE [%]", 
                            formatter=HTMLTemplateFormatter(template=rmse_template)),
                TableColumn(field='r', title='r', 
                            formatter=HTMLTemplateFormatter(template=r_template)),
                TableColumn(field='PBIAS [%]', title='PBIAS [%]', 
                            formatter=HTMLTemplateFormatter(template=PBIAS_template)),
                TableColumn(field='NSE', title='NSE', 
                            formatter=HTMLTemplateFormatter(template=NSE_template)),
        ]
        global_table = DataTable(source=global_src, columns=global_columns, index_position=None, width=bokeh_table_width, height=bokeh_table_height)

        # Title for the table
        table_title = Div(text='<h2>Average Statistics by Simulation</h2>', width=850)
        
        # Title for the table and table combined as one list to make a row
        global_row = [table_title, global_table]
        # Insert this row at the beginning of plots
        plots.insert(0, global_row)


    for key in grouped_data:
       
        # Skip the plot creation if gauge data is not present
        if gauge_csv_file_name not in [series.name for series in grouped_data[key]]:
            continue

        gauge_label = gauge_labels.get(key, "")
        
        # Remaining logic for plotting and stats calculation remains same

    for key in grouped_data:
        gauge_label = gauge_labels.get(key, "")
    
        p = figure(width=bokeh_plot_width, height=bokeh_plot_height, x_axis_type="datetime", title=f"{directory_name}          {gauge_label}          RAS River Station {key[0]} - {key[1]}")

        legend_items = []

        for idx, series in enumerate(grouped_data[key]):
            if gauge_csv_file_name in series.name:  # If the series is derived from Gauge_data.csv
                label_name = series.name.replace(f"{gauge_csv_file_name}_", "").replace(".1", "")
            else:
                label_name = series.name.split('/')[0]  # Extract the file name

            # Create a new data source for each line plot
            source = ColumnDataSource(data=dict(
                x=series.index, 
                y=series.values, 
                label_name=[label_name]*len(series)
            ))

            # Define line properties
            if gauge_csv_file_name in series.name:
                line = p.line(x='x', y='y', source=source, line_width=2, color='black', line_dash='dashed')
            else:
                if base_condition_dss in series.name:
                    line = p.line(x='x', y='y', source=source, line_width=4, color='red', line_dash='dotted')
                else:
                    color = Category20[20][idx % 20]
                    line = p.line(x='x', y='y', source=source, line_width=2, color=color)

            # Add a hover tool for each line
            hover = HoverTool(tooltips=[
                ("Series Name", "@label_name"), 
                ("Date", "@x{%F}"), 
                ("Value", "@y")
            ], formatters={
                "@x": "datetime"
            }, renderers=[line])
            p.add_tools(hover)

            # Add line to the legend
            legend_items.append(LegendItem(label=label_name, renderers=[line]))

            hover = HoverTool(tooltips=[("Series Name", "@label_name"), ("Date", "@x{%F %T}"), ("Value", "@y{0.000}")], formatters={"@x": "datetime"}, renderers=[line])
            p.add_tools(hover)

        legend = Legend(items=legend_items, location="top_right")
        p.add_layout(legend)

        p.xaxis.axis_label = 'Date'
        unit = "CFS" if key[1] == "FLOW" else "FT"
       
        if key[1] == "FLOW":
            p.yaxis.axis_label = 'Discharge (CFS)'
        else:
            p.yaxis.axis_label = 'Stage (FT)'

        #plots.append([p])

        sheet_name = f"{key[0]}_{key[1]}"
        if sheet_name in stats_data:
            stats_df = stats_data[sheet_name]
            stats_src = ColumnDataSource(stats_df)
        
            columns = [
                TableColumn(field="Simulation", title="Simulation",
                            formatter=StringFormatter(font_style="bold", text_align='left'), width=20),
                TableColumn(field="RMSE [%]", title="RMSE [%]", 
                            formatter=HTMLTemplateFormatter(template=rmse_template), width=10),
                TableColumn(field='r', title='r', 
                            formatter=HTMLTemplateFormatter(template=r_template), width=10),
                TableColumn(field='PBIAS [%]', title='PBIAS [%]', 
                            formatter=HTMLTemplateFormatter(template=PBIAS_template), width=10),
                TableColumn(field='NSE', title='NSE', 
                            formatter=HTMLTemplateFormatter(template=NSE_template), width=10),
            ]
            stats_table = DataTable(source=stats_src, columns=columns, index_position=None, editable=False, width=bokeh_table_width, height=bokeh_table_height)
            
            
            # Append the table and plot as separate rows (i.e., separate lists)
            plots.append([p])               # Adding plot in its own row
            plots.append([stats_table])     # Adding table in its own row

        else:
            plots.append([p])  # if stats do not exist just append plot           

    layout = gridplot(plots)
    save(layout)
pass

def calculate_calibration_statistics(DSS_Source_Path):
    # Step 1: Read the input excel file
    input_1_path = os.path.join(DSS_Source_Path, '1_Output_Plotted_Data_Depth.xlsx')
    df = pd.read_excel(input_1_path, sheet_name=None)

    # Step 2: Loop through all the sheets in the input file and calculate the statistics for each column separately
    stats = {}
    for sheet_name, df_sheet in df.items():
        # Split sheet_name on underscore, and take the second part as the parameter value
        parameter = sheet_name.split('_')[1]
        sheet_stats = calculate_stats(df_sheet, parameter)
        stats[sheet_name] = sheet_stats

    # Step 4: Convert the statistics to a dataframe and save to a new excel file with one sheet per sheet in the input file
    with pd.ExcelWriter(os.path.join(DSS_Source_Path, '2_Output_Statistics_Depth.xlsx')) as writer:
        for sheet_name, sheet_stats in stats.items():
            # Skip if there are no statistics for the current sheet
            if len(sheet_stats) == 0:
                continue
            # Define the column names for the statistics DataFrame
            cols = ['Simulation', 'Reach', 'Station', 'Data Type', 'n', 'RMSE [%]', 'r', 'PBIAS [%]', 'NSE']
            # Create a DataFrame from the statistics and write it to the Excel file
            stats_df = pd.DataFrame(sheet_stats, columns=cols)
            stats_df.to_excel(writer, sheet_name=sheet_name, index=False)
                
        print("Statistics file saved successfully.")

    # Load the saved statistics excel file
    output_2_path = os.path.join(DSS_Source_Path, '2_Output_Statistics_Depth.xlsx')
    # Load the Excel workbook
    xls = pd.ExcelFile(output_2_path)
    # Get the names of the sheets in the workbook
    sheet_names = xls.sheet_names

    # Initialize empty dataframes to store cumulative values for the four statistics
    rmse_df = pd.DataFrame()
    r_df = pd.DataFrame()
    pbias_df = pd.DataFrame()
    nse_df = pd.DataFrame()

    # Extract data from sheets and combine
    for sheet_name in sheet_names:
        # Skip the "global summaries" sheet
        if sheet_name != "global summaries":
            # Read the data from the current sheet
            df = pd.read_excel(xls, sheet_name=sheet_name)
            # Group the data by simulation and calculate the mean for each statistic
            rmse_group = df.groupby("Simulation")["RMSE [%]"].mean().reset_index()
            r_group = df.groupby("Simulation")["r"].mean().reset_index()
            pbias_group = df.groupby("Simulation")["PBIAS [%]"].mean().reset_index()
            nse_group = df.groupby("Simulation")["NSE"].mean().reset_index()
            # Append the grouped data to the corresponding DataFrame
            rmse_df = pd.concat([rmse_df, rmse_group], axis=0)
            r_df = pd.concat([r_df, r_group], axis=0)
            pbias_df = pd.concat([pbias_df, pbias_group], axis=0)
            nse_df = pd.concat([nse_df, nse_group], axis=0)

    # Compute averages for each simulation
    rmse_final = rmse_df.groupby("Simulation")["RMSE [%]"].mean().reset_index()
    r_final = r_df.groupby("Simulation")["r"].mean().reset_index()
    pbias_final = pbias_df.groupby("Simulation")["PBIAS [%]"].mean().reset_index()
    nse_final = nse_df.groupby("Simulation")["NSE"].mean().reset_index()

    # Merge the final dataframes for RMSE, r, PBIAS, and NSE based on Simulation name
    global_summaries_df = rmse_final.merge(r_final, on="Simulation", how="outer")\
                                    .merge(pbias_final, on="Simulation", how="outer")\
                                    .merge(nse_final, on="Simulation", how="outer")
    


    # Write the global summaries dataframe to a new sheet in the Excel workbook
    with pd.ExcelWriter(output_2_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        global_summaries_df.to_excel(writer, sheet_name="global summaries", index=False, startcol=4)

    print("Global summaries sheet updated successfully in the Excel workbook.")

    
    # Load all sheets into separate dataframes
    all_dataframes = {sheet: pd.read_excel(output_2_path, sheet_name=sheet) for sheet in sheet_names}

    # Open the Excel writer
    with pd.ExcelWriter(output_2_path, engine='openpyxl') as writer:

        # First, write the global summaries sheet
        global_summaries_df.to_excel(writer, sheet_name="global summaries", index=False, startcol=4)

        # Then, write all other sheets
        for sheet, data in all_dataframes.items():
            # Avoid writing 'global summaries' again
            if sheet != "global summaries":  
                data.to_excel(writer, sheet_name=sheet, index=False)

    print("Excel workbook updated with 'global summaries' as the first sheet.")

def apply_conditional_formatting(output_2_path):
    # Load the workbook for applying conditional formatting
    wb = load_workbook(output_2_path)


    # Define the styles for conditional formatting
    soft_green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
    soft_red_fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
    bold_black_font = Font(bold=True)

    # Define conditional formatting rules
    rmse_green_rule = CellIsRule(operator='between', formula=['0', '15'], stopIfTrue=True, fill=soft_green_fill, font=bold_black_font)
    rmse_red_rule = CellIsRule(operator='between', formula=['15.0001', '100'], stopIfTrue=True, fill=soft_red_fill, font=bold_black_font)

    r_green_rule = CellIsRule(operator='between', formula=['0.9', '1'], stopIfTrue=True, fill=soft_green_fill, font=bold_black_font)
    r_red_rule = CellIsRule(operator='lessThan', formula=['0.9'], stopIfTrue=True, fill=soft_red_fill, font=bold_black_font)

    pbias_green_rule = CellIsRule(operator='between', formula=['-10', '10'], stopIfTrue=True, fill=soft_green_fill, font=bold_black_font)
    pbias_red_rule_gt = CellIsRule(operator='greaterThan', formula=['10'], stopIfTrue=True, fill=soft_red_fill, font=bold_black_font)
    pbias_red_rule_lt = CellIsRule(operator='lessThan', formula=['-10'], stopIfTrue=True, fill=soft_red_fill, font=bold_black_font)

    nse_green_rule = CellIsRule(operator='greaterThanOrEqual', formula=['0.6'], stopIfTrue=True, fill=soft_green_fill, font=bold_black_font)
    nse_red_rule = CellIsRule(operator='lessThan', formula=['0.6'], stopIfTrue=True, fill=soft_red_fill, font=bold_black_font)


    # Apply conditional formatting rules to each sheet
    for sheet_name in wb.sheetnames:
        # Get the current sheet
        ws = wb[sheet_name]
        
        # Calculate the maximum length of the values in column A to adjust the column width
        column_widths = []
        column = ws['A']
        max_length = 0
        column = ws['A']
        for cell in column:
            try: 
                if len(str(cell.value)) > max_length:
                    max_length = len(cell.value)
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions['A'].width = adjusted_width

        # Find the last row with data in the sheet
        last_row = ws.max_row

         # Apply formatting rules for RMSE
        ws.conditional_formatting.add(f'F2:F{last_row}', rmse_green_rule)
        ws.conditional_formatting.add(f'F2:F{last_row}', rmse_red_rule)

        # Apply formatting rules for r
        ws.conditional_formatting.add(f'G2:G{last_row}', r_green_rule)
        ws.conditional_formatting.add(f'G2:G{last_row}', r_red_rule)

        # Apply formatting rules for PBIAS
        ws.conditional_formatting.add(f'H2:H{last_row}', pbias_green_rule)
        ws.conditional_formatting.add(f'H2:H{last_row}', pbias_red_rule_gt)
        ws.conditional_formatting.add(f'H2:H{last_row}', pbias_red_rule_lt)

        # Apply formatting rules for NSE
        ws.conditional_formatting.add(f'I2:I{last_row}', nse_green_rule)
        ws.conditional_formatting.add(f'I2:I{last_row}', nse_red_rule)

        # Set number format for RMSE, r, PBIAS, and NSE columns to display 3 decimal places
        for row in range(2, last_row + 1):
            ws[f'F{row}'].number_format = '0.00'  # RMSE
            ws[f'G{row}'].number_format = '0.00'  # r
            ws[f'H{row}'].number_format = '0.00'  # PBIAS
            ws[f'I{row}'].number_format = '0.00'  # NSE

    # Save the workbook with the applied conditional formatting
    wb.save(output_2_path)
    print("Conditional formatting applied successfully.")    

def read_dss_Data_to_CSV(folder_path):
    # Initialize an empty DataFrame
    df = pd.DataFrame()
    # Loop through all files in the directory
    for file in os.listdir(folder_path):
        # Check if the file is a .dss file
        if file.endswith(".dss"):
            # Open the .dss file
            with HecDss.Open(os.path.join(folder_path, file)) as fid:
                # Get a list of pathnames in the .dss file
                path_list = fid.getPathnameList(pathname_pattern, sort=1)
                # Filter the pathnames based on certain conditions
                filtered_path_list = [
                    path for path in path_list
                    if any(station in path for station in river_stations)
                    and search_word in path
                    and (search_parameter1 in path or search_parameter2 in path)                    
                    and "FLOW-CUM" not in path
                ]
                # Loop through the filtered pathnames
                for pathname in filtered_path_list:
                    # Read the time series data from the .dss file
                    ts = fid.read_ts(pathname, window=(plot_window_start, plot_window_end), trim_missing=True)
                    # Check if the time series data is not None
                    if ts is not None:
                        # Get the times and values from the time series data
                        times, values = np.array(ts.pytimes), ts.values
                        # Construct a header for the DataFrame
                        header = f"{file} {pathname}"
                        # Create a temporary DataFrame with the time series data
                        temp_df = pd.DataFrame({header: pd.Series(values[~ts.nodata], index=times[~ts.nodata])})
                        # Check if the values in the temporary DataFrame are not identical to the existing DataFrame
                        if not check_identical_values(df, temp_df[header]):
                            # Concatenate the temporary DataFrame with the existing DataFrame
                            df = pd.concat([df, temp_df], axis=1)

    # Save the DataFrame to a .csv file
    df.to_csv(os.path.join(folder_path, "0_Output_raw.csv"), index_label="Date")
    return df

def process_folder(folder_path):

    # Get the name of the directory from the folder path
    directory_name = os.path.basename(folder_path)
    # Construct the path to the gauge data file
    gauge_data_path = os.path.join(folder_path, gauge_csv_file_name)
    
    # Call the function read_dss_Data_to_CSV
    df = read_dss_Data_to_CSV(folder_path)

    # Group the data by river station and parameter (STAGE or FLOW)
    grouped_data = {(split_label[2], "STAGE" if "STAGE" in split_label[3] else "FLOW"): [] for column in df.columns for split_label in [column.split('/')]}
    for column in df.columns:
        split_label = column.split('/')
        grouped_data[(split_label[2], "STAGE" if "STAGE" in split_label[3] else "FLOW")].append(df[column])

    # Read the gauge data from the .csv file
    gauge_data = pd.read_csv(gauge_data_path, header=None, nrows=3)
    # Extract the metadata from the gauge data
    metadata = {key: gauge_data.iloc[idx, 1:].tolist() for idx, key in enumerate(["River Station", "STAGE or FLOW", "USGS"])}
    # Read the data from the gauge data
    data = pd.read_csv(gauge_data_path, header=2, parse_dates=["Date"])
    # Set the Date column as the index of the DataFrame
    data.set_index("Date", inplace=True)

    # Loop through the columns in the data
    for idx, column in enumerate(data.columns):
        # Get the key for the grouped data
        key = (metadata["River Station"][idx], metadata["STAGE or FLOW"][idx])
        # Check if the key exists in the grouped data
        if key in grouped_data:
            # Append the column to the grouped data
            grouped_data[key].append(data[column].rename(f"{gauge_csv_file_name}_{column}"))

    # Filter the grouped data to only include series that contain the gauge data
    grouped_data = {k: v for k, v in grouped_data.items() if any(gauge_csv_file_name in s.name for s in v)}
    
    save_plotted_data_to_excel(grouped_data, folder_path)  # Save plotted data (Stage/Flow) to excel
    save_plotted_data_to_excel_depth(grouped_data, folder_path) # Save plotted data (Depth/Flow) to excel

    calculate_calibration_statistics(folder_path)
    apply_conditional_formatting(os.path.join(folder_path, '2_Output_Statistics_Depth.xlsx'))
    
    # Plot the grouped data using Bokeh
    plot_grouped_data_bokeh(grouped_data, folder_path, directory_name)    

In [ ]:
# Iterate over all DSS files in the source directory and sort/add to plots

for root, dirs, files in os.walk(DSS_Source_Path):
    # If the gauge CSV file is found in the current directory, process the folder
    if gauge_csv_file_name in files:
        process_folder(root)